# CLIP을 활용한 음식 Labeling with Colab

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FVGte6%2Fbtq9IEniUZC%2FRyA3CCb2oekVmKAqqgzmK1%2Fimg.png" width = "700" height="300"/>

- CLIP의 전반적인 아키텍처

    - CLIP은 기본적으로 (텍스트, 이미지)가 한 쌍의 input으로 주어지는 모델입니다.  
    따라서 N개의 (텍스트, 이미지) 쌍의 배치가 주어진다고 할 때,  
    CLIP은 그림에서 보시다시피 N*N의 가능한 (텍스트, 이미지) 쌍을  예측하도록 학습됩니다.  
    (그림의 바둑판이 이에 해당합니다.)   
    따라서 만약 대각선에 위치한 실제 (텍스트, 이미지) 쌍이라면, 이들의 코사인 유사도를 최대화 하고,  
    나머지 쌍들은 코사인 유사도를 최소화 하는 방향으로 이미지 인코더와 텍스트 인코더를 함께 학습하게 되는 것입니다. 
    이를 통해 multi-modal 임베딩 공간을 학습하게 됩니다.


<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FOiSFF%2Fbtq9HMlDN53%2FbiLVH3tYzVjxiznSyESX4K%2Fimg.png" width = "600" height="350"/>

위 그림에서 보시다시피 CLIP은 다른 SOTA 모델과 비교했을 때 더 나은 score를 보이고 있음을 확인할 수 있습니다.  
또한 CLIP은 텍스트 간의 연관성 또한 학습하기 때문에 다른 domain의 태스크에도 강력하게 작동한다는 장점이 있습니다.

###### 출처:  https://daeun-computer-uneasy.tistory.com/38

## 1. CLIP을 활용하기 위한 모듈 다운

In [2]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

/bin/bash: conda: command not found
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-9ig529wg
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-9ig529wg


In [6]:
import os
import clip
import torch
from PIL import Image
from torchvision.datasets import CIFAR100

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

In [7]:
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

Files already downloaded and verified


In [ ]:
!unzip -qq "/content/drive/MyDrive/Final_pro/김남규/이미지/_종합_이미지.zip" -d "./dataset"

## 2. 파일들의 경로 및 id값 리스트화

In [48]:
import os

path = '/content/dataset/음식점'
file_list = os.listdir(path)
file_list_img = [file for file in file_list if file.endswith('.png')]
file_list_id = [file[:-4].split("_")[0] for file in file_list if file.endswith('.png')]

In [50]:
file_list[:5]

['517851086_우니담.png',
 '27107805_신의한모.png',
 '167510752_명태명가 제주점.png',
 '9578385_한아름식당.png',
 '1206019377_제주해조네보말성게전문점.png']

In [52]:
file_list_id[:5]

['517851086', '27107805', '167510752', '9578385', '1206019377']

In [82]:
# 데이터 불러오기
import pandas as pd

data = pd.read_excel("/content/drive/MyDrive/Final_pro/김준형/_종합_POI_음식점.xlsx")

data.head(2)

,idx,keyword,address_name,category_group_name,category_name,id,place_name,x,y,rating,kind,content,adjacent_tour,adjacent_rest,adjacent_cafe
0,52,맛집,제주특별자치도 제주시 애월읍 고내리 1105-8,음식점,"음식점 > 한식 > 육류,고기",1580336980,정직한돈 본점,126.338881,33.467358,4.5,제주시,맛집 음식점,22,156,45
1,53,맛집,제주특별자치도 제주시 애월읍 고내리 350,음식점,음식점 > 한식,517851086,우니담,126.348456,33.472497,2.9,제주시,맛집 음식점,20,174,46


In [83]:
data["food_label"] = ""

- CLIP_labeling 함수
    - img : img의 이름
    - data : 제주도 음식점 데이터가 들어있는 DataFrame
    - deviced : cuda인지 cpu인지  

- food_category : CLIP은 영어로 해놓은 카테고리를 통해 음식의 사진을 보고 맞춰가는 과정
    - "chicken", "Grilled pork","chiness food","sea food","noodle","Grilled fish","Grilled Cutlassfish","beef",
    "Hanjeongsik","sashimi","bolied pork","hamburger","shrimp sashimi","shrimp","melon prosciutto","pork cutlet",
    "bread","pizza","Waffle","Tteokbokki","pasta","ramen","sushi","corn dog","American breakfast","crab","curry",
    "bread","soup","tuna","doughnut","koreanstyle sushi(gimbab)"

In [84]:
def CLIP_labeling(img,data,device):
  img_path = Image.open("/content/dataset/음식점/" + img)
  image_input = preprocess(img_path).unsqueeze(0).to(device)

  food_category = ["chicken", "Grilled pork","chiness food","sea food","noodle","Grilled fish","Grilled Cutlassfish","beef","Hanjeongsik",
                 "sashimi","bolied pork","hamburger","shrimp sashimi","shrimp","melon prosciutto","pork cutlet",
                 "bread","pizza","Waffle","Tteokbokki","pasta","ramen","sushi","corn dog","American breakfast","crab","curry","bread","soup","tuna",
                 "doughnut","koreanstyle sushi(gimbab)"]
  
  text_inputs = clip.tokenize(food_category).to(device)

  with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

  image_features /= image_features.norm(dim=-1, keepdim=True)
  text_features /= text_features.norm(dim=-1, keepdim=True)
  similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
  values, indices = similarity[0].topk(5)

  id = int(img[:-4].split("_")[0])
  data.loc[data['id'] == id, 'food_label'] = text_snippets[indices[0]]


In [85]:
for img in file_list_img:
  CLIP_labeling(img,data,device)

data.head(5)

,idx,keyword,address_name,category_group_name,category_name,id,place_name,x,y,rating,kind,content,adjacent_tour,adjacent_rest,adjacent_cafe,food_label
0,52,맛집,제주특별자치도 제주시 애월읍 고내리 1105-8,음식점,"음식점 > 한식 > 육류,고기",1580336980,정직한돈 본점,126.338881,33.467358,4.5,제주시,맛집 음식점,22,156,45,koreanstyle sushi(gimbab)
1,53,맛집,제주특별자치도 제주시 애월읍 고내리 350,음식점,음식점 > 한식,517851086,우니담,126.348456,33.472497,2.9,제주시,맛집 음식점,20,174,46,koreanstyle sushi(gimbab)
2,55,맛집,제주특별자치도 제주시 애월읍 고내리 334-2,음식점,"음식점 > 한식 > 육류,고기",549874234,명리동식당 애월점,126.349180,33.472339,3.4,제주시,맛집 음식점,20,176,46,bolied pork
3,56,맛집,제주특별자치도 제주시 애월읍 고내리 337,음식점,"음식점 > 한식 > 해물,생선 > 회",27505184,남또리횟집,126.349344,33.472809,4.0,제주시,맛집 음식점,20,177,46,sashimi
4,58,맛집,제주특별자치도 제주시 애월읍 고내리 1214,음식점,"음식점 > 한식 > 육류,고기",1266510365,제주돈아 고내점,126.334939,33.466298,4.3,제주시,맛집 음식점,23,150,41,Hanjeongsik


In [86]:
data.to_excel("/content/drive/MyDrive/Final_pro/김준형/_음식점_CLIP라벨링.xlsx")